In [1]:
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import databricks.koalas as ks
import warnings
warnings.filterwarnings("ignore")

import chart_studio.plotly as py
from plotly.offline import plot
import plotly.graph_objs as go
import keras
print(keras.__version__)
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split

# pyoff.init_notebook_mode()

# Read in data
#a.encode('utf-8').strip()
df = ks.read_csv('/FileStore/tables/data.csv')

2.2.5

In [2]:
### EDA ###

print(df.dtypes)

# Change sales date to datetime type
df['Order_Date_Id'] = ks.to_datetime(df['Order_Date_Id'])
print(df.dtypes)
print(df.shape)

Order_Date_Id datetime64[ns]
store int32
item int32
Sales int32
dtype: object
Order_Date_Id datetime64[ns]
store int32
item int32
Sales int32
dtype: object
(65534, 4)

In [3]:
# Change all the dates to first day in the month
df['Order_Date_Id'] = df['Order_Date_Id'].dt.year.astype('str') + '-' + df['Order_Date_Id'].dt.month.astype('str') + '-01'
df['Order_Date_Id'] = ks.to_datetime(df['Order_Date_Id'])

# Compute sales column for each row
# df['Sales'] = df.Sales_Price * df.Quantity

# Groupby date and get sum of sales
df = df.groupby('Order_Date_Id').Sales.sum().reset_index()
print(df)

Order_Date_Id Sales
0 2014-02-01 26506
1 2016-02-01 30688
2 2014-11-01 39640
3 2017-01-01 30442
4 2014-09-01 39084
5 2015-02-01 27260
6 2013-04-01 33896
7 2013-08-01 38391
8 2014-06-01 45607
9 2016-08-01 48727
10 2016-04-01 44864
11 2016-06-01 51039
12 2013-10-01 32611
13 2014-12-01 29196
14 2013-09-01 34164
15 2015-07-01 51422
16 2017-03-01 40610
17 2017-07-01 57785
18 2016-11-01 44339
19 2016-12-01 33296
20 2013-07-01 42478
21 2013-02-01 22994
22 2014-03-01 34989
23 2015-09-01 41331
24 2013-11-01 34828
25 2016-09-01 44687
26 2013-05-01 38073
27 2015-10-01 39704
28 2017-10-01 43554
29 2013-01-01 22738
30 2015-12-01 30279
31 2014-05-01 43776
32 2015-01-01 27694
33 2015-04-01 41168
34 2017-12-01 34435
35 2015-06-01 46979
36 2016-05-01 49376
37 2017-09-01 46036
38 2014-04-01 39160
39 2015-05-01 46279
40 2017-02-01 31123
41 2016-03-01 39418
42 2017-05-01 50878
43 2017-06-01 52830
44 2016-07-01 56394
45 2017-08-01 50232
46 2015-11-01 41430
47 2017-04-01 46889
48 2017-11-01 45707
49 2014-07-01 49130
50 2016-10-01 43079
51 2016-01-01 30094
52 2014-10-01 37684
53 2014-01-01 26072
54 2013-06-01 39602
55 2013-12-01 25359
56 2015-08-01 46178
57 2013-03-01 30722
58 2014-08-01 44028
59 2015-03-01 36288

In [4]:
# Plot monthly sales to check trends
df = df.sort_values(by=['Order_Date_Id'], ascending=True)
df = df.to_spark()
display(df)

Order_Date_Id,Sales
2013-01-01T00:00:00.000+0000,22738
2013-02-01T00:00:00.000+0000,22994
2013-03-01T00:00:00.000+0000,30722
2013-04-01T00:00:00.000+0000,33896
2013-05-01T00:00:00.000+0000,38073
2013-06-01T00:00:00.000+0000,39602
2013-07-01T00:00:00.000+0000,42478
2013-08-01T00:00:00.000+0000,38391
2013-09-01T00:00:00.000+0000,34164
2013-10-01T00:00:00.000+0000,32611


In [5]:
# Create a new dataframe for modeling purposes
df = ks.DataFrame(df)
df_model = df.copy()
print(df_model.shape)

# Add previous sales to next row
df_model['Prev_Sales'] = df_model['Sales'].shift(1)

# Drop the null values and calculate the difference
df_model = df_model.dropna()
df_model['Difference'] = (df_model['Sales'] - df_model['Prev_Sales'])
print(df_model.shape)

(60, 2)
(59, 4)

In [6]:
print(df_model.iloc[-10:])

Order_Date_Id Sales Prev_Sales Difference
50 2017-03-01 40610 31123 9487
51 2017-04-01 46889 40610 6279
52 2017-05-01 50878 46889 3989
53 2017-06-01 52830 50878 1952
54 2017-07-01 57785 52830 4955
55 2017-08-01 50232 57785 -7553
56 2017-09-01 46036 50232 -4196
57 2017-10-01 43554 46036 -2482
58 2017-11-01 45707 43554 2153
59 2017-12-01 34435 45707 -11272

In [7]:
# Plot monthly sales difference
df_model = df_model.sort_values(by=['Order_Date_Id'], ascending=True)
df_model = df_model.to_spark()
display(df_model)

Order_Date_Id,Sales,Prev_Sales,Difference
2013-02-01T00:00:00.000+0000,22994,22738,256
2013-03-01T00:00:00.000+0000,30722,22994,7728
2013-04-01T00:00:00.000+0000,33896,30722,3174
2013-05-01T00:00:00.000+0000,38073,33896,4177
2013-06-01T00:00:00.000+0000,39602,38073,1529
2013-07-01T00:00:00.000+0000,42478,39602,2876
2013-08-01T00:00:00.000+0000,38391,42478,-4087
2013-09-01T00:00:00.000+0000,34164,38391,-4227
2013-10-01T00:00:00.000+0000,32611,34164,-1553
2013-11-01T00:00:00.000+0000,34828,32611,2217


In [8]:
# Create dataframe for transformations
df_model = ks.DataFrame(df_model)
df_transformed = df_model.drop(['Prev_Sales'],axis=1)

# Adding time-steps (lag)
for i in range(1,13):
    field_name = 'lag_' + str(i)
    df_transformed[field_name] = df_transformed['Difference'].shift(i)

# Drop null values
df_transformed = df_transformed.dropna().reset_index(drop=True)
print(df_transformed)

Order_Date_Id Sales Difference lag_1 lag_2 lag_3 lag_4 lag_5 lag_6 lag_7 lag_8 lag_9 lag_10 lag_11 lag_12
0 2014-02-01 26506 434 713 -9469 2217 -1553 -4227 -4087 2876 1529 4177 3174 7728 256
1 2014-03-01 34989 8483 434 713 -9469 2217 -1553 -4227 -4087 2876 1529 4177 3174 7728
2 2014-04-01 39160 4171 8483 434 713 -9469 2217 -1553 -4227 -4087 2876 1529 4177 3174
3 2014-05-01 43776 4616 4171 8483 434 713 -9469 2217 -1553 -4227 -4087 2876 1529 4177
4 2014-06-01 45607 1831 4616 4171 8483 434 713 -9469 2217 -1553 -4227 -4087 2876 1529
5 2014-07-01 49130 3523 1831 4616 4171 8483 434 713 -9469 2217 -1553 -4227 -4087 2876
6 2014-08-01 44028 -5102 3523 1831 4616 4171 8483 434 713 -9469 2217 -1553 -4227 -4087
7 2014-09-01 39084 -4944 -5102 3523 1831 4616 4171 8483 434 713 -9469 2217 -1553 -4227
8 2014-10-01 37684 -1400 -4944 -5102 3523 1831 4616 4171 8483 434 713 -9469 2217 -1553
9 2014-11-01 39640 1956 -1400 -4944 -5102 3523 1831 4616 4171 8483 434 713 -9469 2217
10 2014-12-01 29196 -10444 1956 -1400 -4944 -5102 3523 1831 4616 4171 8483 434 713 -9469
11 2015-01-01 27694 -1502 -10444 1956 -1400 -4944 -5102 3523 1831 4616 4171 8483 434 713
12 2015-02-01 27260 -434 -1502 -10444 1956 -1400 -4944 -5102 3523 1831 4616 4171 8483 434
13 2015-03-01 36288 9028 -434 -1502 -10444 1956 -1400 -4944 -5102 3523 1831 4616 4171 8483
14 2015-04-01 41168 4880 9028 -434 -1502 -10444 1956 -1400 -4944 -5102 3523 1831 4616 4171
15 2015-05-01 46279 5111 4880 9028 -434 -1502 -10444 1956 -1400 -4944 -5102 3523 1831 4616
16 2015-06-01 46979 700 5111 4880 9028 -434 -1502 -10444 1956 -1400 -4944 -5102 3523 1831
17 2015-07-01 51422 4443 700 5111 4880 9028 -434 -1502 -10444 1956 -1400 -4944 -5102 3523
18 2015-08-01 46178 -5244 4443 700 5111 4880 9028 -434 -1502 -10444 1956 -1400 -4944 -5102
19 2015-09-01 41331 -4847 -5244 4443 700 5111 4880 9028 -434 -1502 -10444 1956 -1400 -4944
20 2015-10-01 39704 -1627 -4847 -5244 4443 700 5111 4880 9028 -434 -1502 -10444 1956 -1400
21 2015-11-01 41430 1726 -1627 -4847 -5244 4443 700 5111 4880 9028 -434 -1502 -10444 1956
22 2015-12-01 30279 -11151 1726 -1627 -4847 -5244 4443 700 5111 4880 9028 -434 -1502 -10444
23 2016-01-01 30094 -185 -11151 1726 -1627 -4847 -5244 4443 700 5111 4880 9028 -434 -1502
24 2016-02-01 30688 594 -185 -11151 1726 -1627 -4847 -5244 4443 700 5111 4880 9028 -434
25 2016-03-01 39418 8730 594 -185 -11151 1726 -1627 -4847 -5244 4443 700 5111 4880 9028
26 2016-04-01 44864 5446 8730 594 -185 -11151 1726 -1627 -4847 -5244 4443 700 5111 4880
27 2016-05-01 49376 4512 5446 8730 594 -185 -11151 1726 -1627 -4847 -5244 4443 700 5111
28 2016-06-01 51039 1663 4512 5446 8730 594 -185 -11151 1726 -1627 -4847 -5244 4443 700
29 2016-07-01 56394 5355 1663 4512 5446 8730 594 -185 -11151 1726 -1627 -4847 -5244 4443
30 2016-08-01 48727 -7667 5355 1663 4512 5446 8730 594 -185 -11151 1726 -1627 -4847 -5244
31 2016-09-01 44687 -4040 -7667 5355 1663 4512 5446 8730 594 -185 -11151 1726 -1627 -4847
32 2016-10-01 43079 -1608 -4040 -7667 5355 1663 4512 5446 8730 594 -185 -11151 1726 -1627
33 2016-11-01 44339 1260 -1608 -4040 -7667 5355 1663 4512 5446 8730 594 -185 -11151 1726
34 2016-12-01 33296 -11043 1260 -1608 -4040 -7667 5355 1663 4512 5446 8730 594 -185 -11151
35 2017-01-01 30442 -2854 -11043 1260 -1608 -4040 -7667 5355 1663 4512 5446 8730 594 -185
36 2017-02-01 31123 681 -2854 -11043 1260 -1608 -4040 -7667 5355 1663 4512 5446 8730 594
37 2017-03-01 40610 9487 681 -2854 -11043 1260 -1608 -4040 -7667 5355 1663 4512 5446 8730
38 2017-04-01 46889 6279 9487 681 -2854 -11043 1260 -1608 -4040 -7667 5355 1663 4512 5446
39 2017-05-01 50878 3989 6279 9487 681 -2854 -11043 1260 -1608 -4040 -7667 5355 1663 4512
40 2017-06-01 52830 1952 3989 6279 9487 681 -2854 -11043 1260 -1608 -4040 -7667 5355 1663
41 2017-07-01 57785 4955 1952 3989 6279 9487 681 -2854 -11043 1260 -1608 -4040 -7667 5355
42 2017-08-01 50232 -7553 4955 1952 3989 6279 9487 681 -2854 -11043 1260 -1608 -4040 -7667
43 2017-09-01 46036 -4196 -7553 4955 1952 3989 6279 9487 681 -2854 

In [9]:
df_transformed.head(20)

,Order_Date_Id,Sales,Difference,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
0,2014-02-01,26506,434,713,-9469,2217,-1553,-4227,-4087,2876,1529,4177,3174,7728,256
1,2014-03-01,34989,8483,434,713,-9469,2217,-1553,-4227,-4087,2876,1529,4177,3174,7728
2,2014-04-01,39160,4171,8483,434,713,-9469,2217,-1553,-4227,-4087,2876,1529,4177,3174
3,2014-05-01,43776,4616,4171,8483,434,713,-9469,2217,-1553,-4227,-4087,2876,1529,4177
4,2014-06-01,45607,1831,4616,4171,8483,434,713,-9469,2217,-1553,-4227,-4087,2876,1529
5,2014-07-01,49130,3523,1831,4616,4171,8483,434,713,-9469,2217,-1553,-4227,-4087,2876
6,2014-08-01,44028,-5102,3523,1831,4616,4171,8483,434,713,-9469,2217,-1553,-4227,-4087
7,2014-09-01,39084,-4944,-5102,3523,1831,4616,4171,8483,434,713,-9469,2217,-1553,-4227
8,2014-10-01,37684,-1400,-4944,-5102,3523,1831,4616,4171,8483,434,713,-9469,2217,-1553
9,2014-11-01,39640,1956,-1400,-4944,-5102,3523,1831,4616,4171,8483,434,713,-9469,2217


In [10]:
print(df_transformed.iloc[-5:])

Order_Date_Id Sales Difference lag_1 lag_2 lag_3 lag_4 lag_5 lag_6 lag_7 lag_8 lag_9 lag_10 lag_11 lag_12
42 2017-08-01 50232 -7553 4955 1952 3989 6279 9487 681 -2854 -11043 1260 -1608 -4040 -7667
43 2017-09-01 46036 -4196 -7553 4955 1952 3989 6279 9487 681 -2854 -11043 1260 -1608 -4040
44 2017-10-01 43554 -2482 -4196 -7553 4955 1952 3989 6279 9487 681 -2854 -11043 1260 -1608
45 2017-11-01 45707 2153 -2482 -4196 -7553 4955 1952 3989 6279 9487 681 -2854 -11043 1260
46 2017-12-01 34435 -11272 2153 -2482 -4196 -7553 4955 1952 3989 6279 9487 681 -2854 -11043

In [11]:
# import MinMaxScaler, create a new dataframe for LSTM model
from sklearn.preprocessing import MinMaxScaler
df_lstm = df_transformed.drop(['Sales','Order_Date_Id'], axis=1)

# Train/test split
train_set, test_set = df_lstm[0:-6].values, df_lstm[-6:].values
print(train_set.shape)

(41, 13)

In [12]:
# apply scaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)

# Reshape training set
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_scaled = scaler.transform(train_set)

# Reshape test set
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_scaled = scaler.transform(test_set)
print(test_set.shape)

(6, 13)

In [13]:
# Train and test (x = lags, y = difference)
X_train, y_train = train_scaled[:, 1:], train_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test, y_test = test_scaled[:, 1:], test_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
print(X_train.shape)

(41, 1, 12)

In [14]:
# Build model
model = Sequential()
model.add(LSTM(4, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, nb_epoch=100, batch_size=1, verbose=1, shuffle=False)

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1020: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3005: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

Epoch 1/100
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:197: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:207: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:216: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:223: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.

 1/41 [..............................] - ETA: 1:08 - loss: 0.011819/41 [============>.................] - ETA: 2s - loss: 0.1452 37/41 [==========================>...] - ETA: 0s - loss: 0.175541/41 [==============================] - 2s 45ms/step - loss: 0.1732
Epoch 2/100
 1/41 [..............................] - ETA: 0s - loss: 0.006418/41 [============>.................] - ETA: 0s - loss: 0.127336/41 [=========================>....] - ETA: 0s - loss: 0.161741/41 [==============================] - 0s 3ms/step - loss: 0.1573
Epoch 3/100
 1/41 [..............................] - ETA: 0s - loss: 0.006319/41 [============>.................] - ETA: 0s - loss: 0.116737/41 [==========================>...] - ETA: 0s - loss: 0.1469

In [15]:
y_pred = model.predict(X_test,batch_size=1)
print(y_pred)

[[ 0.6354188 ]
 [-0.5072942 ]
 [-0.38257998]
 [ 0.01869548]
 [ 0.2606343 ]
 [-0.9959355 ]]

In [16]:
### Predictions are scaled values, must be inversely scaled to original ###

# Reshape y_pred
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])

# Rebuild test set for inverse transform
pred_test_set = []
for index in range(0, len(y_pred)):
    pred_test_set.append(np.concatenate([y_pred[index], X_test[index]], axis=1))

# Reshape pred_test_set
pred_test_set = np.array(pred_test_set)
pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])
print(pred_test_set.shape)

# Inverse transform
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)
print(pred_test_set_inverted)

(6, 13)
[[ 5724.88631546 1952. 3989. 6279.
 9487. 681. -2854. -11043.
 1260. -1608. -4040. -7667.
 5355. ]
 [ -6066.76862288 4955. 1952. 3989.
 6279. 9487. 681. -2854.
 -11043. 1260. -1608. -4040.
 -7667. ]
 [ -4779.84283715 -7553. 4955. 1952.
 3989. 6279. 9487. 681.
 -2854. -11043. 1260. -1608.
 -4040. ]
 [ -639.08131109 -4196. -7553. 4955.
 1952. 3989. 6279. 9487.
 681. -2854. -11043. 1260.
 -1608. ]
 [ 1857.48537362 -2482. -4196. -7553.
 4955. 1952. 3989. 6279.
 9487. 681. -2854. -11043.
 1260. ]
 [-11109.05842108 2153. -2482. -4196.
 -7553. 4955. 1952. 3989.
 6279. 9487. 681. -2854.
 -11043. ]]

In [17]:
# Create dataframe that shows the predicted sales
result_list = []
sales_dates = list(df[-7:].Order_Date_Id.to_numpy())
print(sales_dates)
act_sales = list(df[-7:].Sales.to_numpy())
print(act_sales)
for index in range(0,len(pred_test_set_inverted)):
    result_dict = {}
    result_dict['Pred_Value'] = int(pred_test_set_inverted[index][0] + act_sales[index])
    result_dict['Order_Date_Id'] = sales_dates[index+1]
    result_list.append(result_dict)
df_result = ks.DataFrame(result_list)

[numpy.datetime64('2017-06-01T00:00:00.000000000'), numpy.datetime64('2017-07-01T00:00:00.000000000'), numpy.datetime64('2017-08-01T00:00:00.000000000'), numpy.datetime64('2017-09-01T00:00:00.000000000'), numpy.datetime64('2017-10-01T00:00:00.000000000'), numpy.datetime64('2017-11-01T00:00:00.000000000'), numpy.datetime64('2017-12-01T00:00:00.000000000')]
[52830, 57785, 50232, 46036, 43554, 45707, 34435]

In [18]:
print(df_result)

Order_Date_Id Pred_Value
0 2017-07-01 58554
1 2017-08-01 51718
2 2017-09-01 45452
3 2017-10-01 45396
4 2017-11-01 45411
5 2017-12-01 34597

In [19]:
# Merge with actual sales dataframe
df_sales_pred = ks.merge(df,df_result,on='Order_Date_Id',how='left')
df_sales_pred = df_sales_pred.sort_values(by=['Order_Date_Id'], ascending=True)
df_sales_pred = df_sales_pred.reset_index()
df_sales_pred = df_sales_pred.drop(columns=['index'])
print(df_sales_pred.iloc[-10:])

Order_Date_Id Sales Pred_Value
50 2017-03-01 40610 NaN
51 2017-04-01 46889 NaN
52 2017-05-01 50878 NaN
53 2017-06-01 52830 NaN
54 2017-07-01 57785 58554.0
55 2017-08-01 50232 51718.0
56 2017-09-01 46036 45452.0
57 2017-10-01 43554 45396.0
58 2017-11-01 45707 45411.0
59 2017-12-01 34435 34597.0

In [20]:
# Plot actual and predicted
df_sales_pred = df_sales_pred.to_spark()
display(df_sales_pred)

Order_Date_Id,Sales,Pred_Value
2013-01-01T00:00:00.000+0000,22738,null
2013-02-01T00:00:00.000+0000,22994,null
2013-03-01T00:00:00.000+0000,30722,null
2013-04-01T00:00:00.000+0000,33896,null
2013-05-01T00:00:00.000+0000,38073,null
2013-06-01T00:00:00.000+0000,39602,null
2013-07-01T00:00:00.000+0000,42478,null
2013-08-01T00:00:00.000+0000,38391,null
2013-09-01T00:00:00.000+0000,34164,null
2013-10-01T00:00:00.000+0000,32611,null


In [21]:
### Future predictions ###
df = ks.DataFrame(df_sales_pred)

In [22]:
print(pred_test_set_inverted)

[[ 5724.88631546 1952. 3989. 6279.
 9487. 681. -2854. -11043.
 1260. -1608. -4040. -7667.
 5355. ]
 [ -6066.76862288 4955. 1952. 3989.
 6279. 9487. 681. -2854.
 -11043. 1260. -1608. -4040.
 -7667. ]
 [ -4779.84283715 -7553. 4955. 1952.
 3989. 6279. 9487. 681.
 -2854. -11043. 1260. -1608.
 -4040. ]
 [ -639.08131109 -4196. -7553. 4955.
 1952. 3989. 6279. 9487.
 681. -2854. -11043. 1260.
 -1608. ]
 [ 1857.48537362 -2482. -4196. -7553.
 4955. 1952. 3989. 6279.
 9487. 681. -2854. -11043.
 1260. ]
 [-11109.05842108 2153. -2482. -4196.
 -7553. 4955. 1952. 3989.
 6279. 9487. 681. -2854.
 -11043. ]]

In [23]:
# Make list from transformed df columns
col_list = list(df_transformed.columns)
col_exclude = ('Order_Date_Id', 'Sales')

fin_col = [item for item in col_list if item not in col_exclude]
print(fin_col)

['Difference', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11', 'lag_12']

In [24]:
df1 = ks.DataFrame(pred_test_set_inverted, columns = fin_col)
print(df1)

Difference lag_1 lag_2 lag_3 lag_4 lag_5 lag_6 lag_7 lag_8 lag_9 lag_10 lag_11 lag_12
0 5724.886315 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0 1260.0 -1608.0 -4040.0 -7667.0 5355.0
1 -6066.768623 4955.0 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0 1260.0 -1608.0 -4040.0 -7667.0
2 -4779.842837 -7553.0 4955.0 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0 1260.0 -1608.0 -4040.0
3 -639.081311 -4196.0 -7553.0 4955.0 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0 1260.0 -1608.0
4 1857.485374 -2482.0 -4196.0 -7553.0 4955.0 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0 1260.0
5 -11109.058421 2153.0 -2482.0 -4196.0 -7553.0 4955.0 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0

In [25]:
# Add predicted order dates and sales
future_df = ks.concat([df_result, df1], axis = 1)
future_df = future_df.sort_values(by=['Order_Date_Id'], ascending=True)
future_df = future_df.reset_index()
future_df = future_df.drop(columns=['index'])
print(future_df.iloc[-10:])

Order_Date_Id Pred_Value Difference lag_1 lag_2 lag_3 lag_4 lag_5 lag_6 lag_7 lag_8 lag_9 lag_10 lag_11 lag_12
0 2017-07-01 58554 5724.886315 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0 1260.0 -1608.0 -4040.0 -7667.0 5355.0
1 2017-08-01 51718 -6066.768623 4955.0 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0 1260.0 -1608.0 -4040.0 -7667.0
2 2017-09-01 45452 -4779.842837 -7553.0 4955.0 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0 1260.0 -1608.0 -4040.0
3 2017-10-01 45396 -639.081311 -4196.0 -7553.0 4955.0 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0 1260.0 -1608.0
4 2017-11-01 45411 1857.485374 -2482.0 -4196.0 -7553.0 4955.0 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0 1260.0
5 2017-12-01 34597 -11109.058421 2153.0 -2482.0 -4196.0 -7553.0 4955.0 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0

In [26]:
# Merge with main transformed df
print(df_transformed.shape)
df_transformed2 = df_transformed.iloc[:-6]
future_df = future_df.rename(columns={'Pred_Value': 'Sales'})
main_df = ks.concat([df_transformed2, future_df], axis = 0, ignore_index = True)

(47, 15)

In [27]:
print(main_df.iloc[-10:])

Order_Date_Id Sales Difference lag_1 lag_2 lag_3 lag_4 lag_5 lag_6 lag_7 lag_8 lag_9 lag_10 lag_11 lag_12
37 2017-03-01 40610 9487.000000 681.0 -2854.0 -11043.0 1260.0 -1608.0 -4040.0 -7667.0 5355.0 1663.0 4512.0 5446.0 8730.0
38 2017-04-01 46889 6279.000000 9487.0 681.0 -2854.0 -11043.0 1260.0 -1608.0 -4040.0 -7667.0 5355.0 1663.0 4512.0 5446.0
39 2017-05-01 50878 3989.000000 6279.0 9487.0 681.0 -2854.0 -11043.0 1260.0 -1608.0 -4040.0 -7667.0 5355.0 1663.0 4512.0
40 2017-06-01 52830 1952.000000 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0 1260.0 -1608.0 -4040.0 -7667.0 5355.0 1663.0
41 2017-07-01 58554 5724.886315 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0 1260.0 -1608.0 -4040.0 -7667.0 5355.0
42 2017-08-01 51718 -6066.768623 4955.0 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0 1260.0 -1608.0 -4040.0 -7667.0
43 2017-09-01 45452 -4779.842837 -7553.0 4955.0 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0 1260.0 -1608.0 -4040.0
44 2017-10-01 45396 -639.081311 -4196.0 -7553.0 4955.0 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0 1260.0 -1608.0
45 2017-11-01 45411 1857.485374 -2482.0 -4196.0 -7553.0 4955.0 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0 1260.0
46 2017-12-01 34597 -11109.058421 2153.0 -2482.0 -4196.0 -7553.0 4955.0 1952.0 3989.0 6279.0 9487.0 681.0 -2854.0 -11043.0

In [28]:
# Move window forward x rows
main_df = main_df.iloc[2:]
main_df = main_df.reset_index()
print(main_df.head(5))

index Order_Date_Id Sales Difference lag_1 lag_2 lag_3 lag_4 lag_5 lag_6 lag_7 lag_8 lag_9 lag_10 lag_11 lag_12
0 2 2014-04-01 39160 4171.0 8483.0 434.0 713.0 -9469.0 2217.0 -1553.0 -4227.0 -4087.0 2876.0 1529.0 4177.0 3174.0
1 3 2014-05-01 43776 4616.0 4171.0 8483.0 434.0 713.0 -9469.0 2217.0 -1553.0 -4227.0 -4087.0 2876.0 1529.0 4177.0
2 4 2014-06-01 45607 1831.0 4616.0 4171.0 8483.0 434.0 713.0 -9469.0 2217.0 -1553.0 -4227.0 -4087.0 2876.0 1529.0
3 5 2014-07-01 49130 3523.0 1831.0 4616.0 4171.0 8483.0 434.0 713.0 -9469.0 2217.0 -1553.0 -4227.0 -4087.0 2876.0
4 6 2014-08-01 44028 -5102.0 3523.0 1831.0 4616.0 4171.0 8483.0 434.0 713.0 -9469.0 2217.0 -1553.0 -4227.0 -4087.0

In [29]:
### Reiterate preprocessing done in training ###
future_feat = main_df.drop(['Order_Date_Id', 'Sales', 'index'], axis = 1)
future_feat2 = future_feat.to_numpy()
print(future_feat2.shape)

# future_feat = future_feat.reshape(-1, 1)

scaler2 = MinMaxScaler(feature_range=(-1, 1))
future_feat = scaler2.fit_transform(future_feat2)
print(future_feat.shape)

(45, 13)
(45, 13)

In [30]:
# # Reshape x
# look_back = 30

future_feat = future_feat.reshape(future_feat.shape[0], 1, future_feat.shape[1])
print(future_feat.shape)

(45, 1, 13)

In [31]:
pred_fut_set = np.array(future_feat)
pred_fut_set_copy = pred_fut_set
print(pred_fut_set_copy.shape)

(45, 1, 13)

In [32]:
num_predictions = 6

for i in range(0, num_predictions):
    currentStep = pred_fut_set_copy[-1:,:,1:] #last step from the previous prediction
    print(currentStep)
    pred_value = model.predict(currentStep, batch_size = 1) # prediction
    # Adding time-steps (lag)
    pred_list = pred_value.tolist()[0]
    for i in range(1,13):
        pred_list.append(pred_fut_set_copy[-i,:,0].tolist()[0]) # append 1st value from each step of array
    print(pred_list)
    pred_array = np.array([[pred_list]])
    pred_fut_set_copy = np.concatenate((pred_fut_set_copy, pred_array), axis=0)

[[[ 0.28927222 -0.15989922 -0.32600058 -0.6513228 0.56081016
 0.26979358 0.46719643 0.68911716 1. 0.1727043
 -0.17765994 -0.9892958 ]]]
[-1.8511394262313843, -0.9959354996681213, 0.26063430309295654, 0.01869548298418522, -0.38257998228073126, -0.5072941780090332, 0.6354187726974487, 0.26979358464967534, 0.4671964337629615, 0.6891171625157476, 1.0, 0.14662273476112025, -0.19594921988564784]
[[[-0.9959355 0.2606343 0.01869548 -0.38257998 -0.50729418
 0.63541877 0.26979358 0.46719643 0.68911716 1.
 0.14662273 -0.19594922]]]
[-1.3955055475234985, -1.8511394262313843, -0.9959354996681213, 0.26063430309295654, 0.01869548298418522, -0.38257998228073126, -0.5072941780090332, 0.6354187726974487, 0.26979358464967534, 0.4671964337629615, 0.6891171625157476, 1.0, 0.14662273476112025]
[[[-1.85113943 -0.9959355 0.2606343 0.01869548 -0.38257998
 -0.50729418 0.63541877 0.26979358 0.46719643 0.68911716
 1. 0.14662273]]]
[-0.256554514169693, -1.3955055475234985, -1.8511394262313843, -0.9959354996681213, 0.26063430309295654, 0.01869548298418522, -0.38257998228073126, -0.5072941780090332, 0.6354187726974487, 0.26979358464967534, 0.4671964337629615, 0.6891171625157476, 1.0]
[[[-1.39550555 -1.85113943 -0.9959355 0.2606343 0.01869548
 -0.38257998 -0.50729418 0.63541877 0.26979358 0.46719643
 0.68911716 1. ]]]
[0.2698659598827362, -0.256554514169693, -1.3955055475234985, -1.8511394262313843, -0.9959354996681213, 0.26063430309295654, 0.01869548298418522, -0.38257998228073126, -0.5072941780090332, 0.6354187726974487, 0.26979358464967534, 0.4671964337629615, 0.6891171625157476]
[[[-0.25655451 -1.39550555 -1.85113943 -0.9959355 0.2606343
 0.01869548 -0.38257998 -0.50729418 0.63541877 0.26979358
 0.46719643 0.68911716]]]
[0.6819428205490112, 0.2698659598827362, -0.256554514169693, -1.3955055475234985, -1.8511394262313843, -0.9959354996681213, 0.26063430309295654, 0.01869548298418522, -0.38257998228073126, -0.5072941780090332, 0.6354187726974487, 0.26979358464967534, 0.4671964337629615]
[[[ 0.26986596 -0.25655451 -1.39550555 -1.85113943 -0.9959355
 0.2606343 0.01869548 -0.38257998 -0.50729418 0.63541877
 0.26979358 0.46719643]]]
[0.675980806350708, 0.6819428205490112, 0.2698659598827362, -0.256554514169693, -1.3955055475234985, -1.8511394262313843, -0.9959354996681213, 0.26063430309295654, 0.01869548298418522, -0.38257998228073126, -0.5072941780090332, 0.6354187726974487, 0.26979358464967534]

In [33]:
pred_fut_set_copy.shape

Out[34]: (51, 1, 13)

In [34]:
### Predictions are scaled values, must be inversely scaled to original ###
# Reshape pred_fut_set_copy
pred_fut_set_copy2 = pred_fut_set_copy
pred_fut_set_copy2 = pred_fut_set_copy2.reshape(pred_fut_set_copy2.shape[0], pred_fut_set_copy2.shape[2])
print(pred_fut_set_copy2.shape)

predicted_diff = pred_fut_set_copy2[pred_fut_set_copy2.shape[0]-num_predictions:,:]
print(predicted_diff.shape)

# Inverse transform
predicted_diff = scaler.inverse_transform(predicted_diff)
print(predicted_diff)

(51, 13)
(6, 13)
[[-19933.90773928 -11109.05842108 1857.48537362 -639.08131109
 -4921.54073122 -6179.84460902 5349.55770713 1660.58237232
 3652.27841845 5891.3476112 9028. 417.85008237
 -3038.52965404]
 [-15232.22174489 -19933.90773928 -11109.05842108 1857.48537362
 -872.87192443 -4921.54073122 -6179.84460902 5349.55770713
 1660.58237232 3652.27841845 5891.3476112 9028.
 417.85008237]
 [ -3479.38603172 -15232.22174489 -19933.90773928 -11109.05842108
 1568.16980106 -872.87192443 -4921.54073122 -6179.84460902
 5349.55770713 1660.58237232 3652.27841845 5891.3476112
 9028. ]
 [ 1952.74684003 -3479.38603172 -15232.22174489 -19933.90773928
 -11109.9912239 1568.16980106 -872.87192443 -4921.54073122
 -6179.84460902 5349.55770713 1660.58237232 3652.27841845
 5891.3476112 ]
 [ 6204.96796525 1952.74684003 -3479.38603172 -15232.22174489
 -19738.57124096 -11109.9912239 1568.16980106 -872.87192443
 -4921.54073122 -6179.84460902 5349.55770713 1660.58237232
 3652.27841845]
 [ 6143.44594073 6204.96796525 1952.74684003 -3479.38603172
 -15141.45322174 -19738.57124096 -11109.9912239 1568.16980106
 -872.87192443 -4921.54073122 -6179.84460902 5349.55770713
 1660.58237232]]

In [35]:
whole_df = np.concatenate((future_feat2, predicted_diff), axis=0)

In [36]:
print(whole_df)

[[ 4171. 8483. 434. 713.
 -9469. 2217. -1553. -4227.
 -4087. 2876. 1529. 4177.
 3174. ]
 [ 4616. 4171. 8483. 434.
 713. -9469. 2217. -1553.
 -4227. -4087. 2876. 1529.
 4177. ]
 [ 1831. 4616. 4171. 8483.
 434. 713. -9469. 2217.
 -1553. -4227. -4087. 2876.
 1529. ]
 [ 3523. 1831. 4616. 4171.
 8483. 434. 713. -9469.
 2217. -1553. -4227. -4087.
 2876. ]
 [ -5102. 3523. 1831. 4616.
 4171. 8483. 434. 713.
 -9469. 2217. -1553. -4227.
 -4087. ]
 [ -4944. -5102. 3523. 1831.
 4616. 4171. 8483. 434.
 713. -9469. 2217. -1553.
 -4227. ]
 [ -1400. -4944. -5102. 3523.
 1831. 4616. 4171. 8483.
 434. 713. -9469. 2217.
 -1553. ]
 [ 1956. -1400. -4944. -5102.
 3523. 1831. 4616. 4171.
 8483. 434. 713. -9469.
 2217. ]
 [-10444. 1956. -1400. -4944.
 -5102. 3523. 1831. 4616.
 4171. 8483. 434. 713.
 -9469. ]
 [ -1502. -10444. 1956. -1400.
 -4944. -5102. 3523. 1831.
 4616. 4171. 8483. 434.
 713. ]
 [ -434. -1502. -10444. 1956.
 -1400. -4944. -5102. 3523.
 1831. 4616. 4171. 8483.
 434. ]
 [ 9028. -434. -1502. -10444.
 1956. -1400. -4944. -5102.
 3523. 1831. 4616. 4171.
 8483. ]
 [ 4880. 9028. -434. -1502.
 -10444. 1956. -1400. -4944.
 -5102. 3523. 1831. 4616.
 4171. ]
 [ 5111. 4880. 9028. -434.
 -1502. -10444. 1956. -1400.
 -4944. -5102. 3523. 1831.
 4616. ]
 [ 700. 5111. 4880. 9028.
 -434. -1502. -10444. 1956.
 -1400. -4944. -5102. 3523.
 1831. ]
 [ 4443. 700. 5111. 4880.
 9028. -434. -1502. -10444.
 1956. -1400. -4944. -5102.
 3523. ]
 [ -5244. 4443. 700. 5111.
 4880. 9028. -434. -1502.
 -10444. 1956. -1400. -4944.
 -5102. ]
 [ -4847. -5244. 4443. 700.
 5111. 4880. 9028. -434.
 -1502. -10444. 1956. -1400.
 -4944. ]
 [ -1627. -4847. -5244. 4443.
 700. 5111. 4880. 9028.
 -434. -1502. -10444. 1956.
 -1400. ]
 [ 1726. -1627. -4847. -5244.
 4443. 700. 5111. 4880.
 9028. -434. -1502. -10444.
 1956. ]
 [-11151. 1726. -1627. -4847.
 -5244. 4443. 700. 5111.
 4880. 9028. -434. -1502.
 -10444. ]
 [ -185. -11151. 1726. -1627.
 -4847. -5244. 4443. 700.
 5111. 4880. 9028. -434.
 -1502. ]
 [ 594. -185. -11151. 1726.
 -1627. -4847. -5244. 4443.
 700. 5111. 4880. 9028.
 -434. ]
 [ 8730. 594. -185. -11151.
 1726. -1627. -4847. -5244.
 4443. 700. 5111. 4880.
 9028. ]
 [ 5446. 8730. 594. -185.
 -11151. 1726. -1627. -4847.
 -5244. 4443. 700. 5111.
 4880. ]
 [ 4512. 5446. 8730. 594.
 -185. -11151. 1726. -1627.
 -4847. -5244. 4443. 700.
 5111. ]
 [ 1663. 4512. 5446. 8730.
 594. -185. -11151. 1726.
 -1627. -4847. -5244. 4443.
 700. ]
 [ 5355. 1663. 4512. 5446.
 8730. 594. -185. -11151.
 1726. -1627. -4847. -5244.
 4443. ]
 [ -7667. 5355. 1663. 4512.
 5446. 8730. 594. -185.
 -11151. 1726. -1627. -4847.
 -5244. ]
 [ -4040. -7667. 5355. 1663.
 4512. 5446. 8730. 594.
 -185. -11151. 1726. -1627.
 -4847. ]
 [ -1608. -4040. -7667. 5355.
 1663. 4512. 5446. 8730.
 594. -185. -11151. 1726.
 -1627. ]
 [ 1260. -1608. -4040. -7667.
 5355. 1663. 4512. 5446.
 8730. 594. -185. -11151.
 1726. ]
 [-11043. 1260. -1608. -4040.
 -7667. 5355. 1663. 4512.
 5446. 8730. 594. -185.
 -11151. ]
 [ -2854. -11043. 1260. -1608.
 -4040. -7667. 5355. 1663.
 4512. 5446. 8730. 594.
 -185. ]
 [ 681. -2854. -11043. 1260.
 -1608. -4040. -7667. 5355.
 1663. 4512. 5446. 8730.
 594. ]
 [ 9487. 681. -2854. -11043.
 1260. -1608. -4040. -7667.
 5355. 1663. 4512. 5446.
 8730. ]
 [ 6279. 9487. 681. -2854.
 -11043. 1260. -1608. -4040.
 -7667. 5355. 1663. 4512.
 5446. ]
 [ 3989. 6279. 9487. 681.
 -2854. -11043. 1260. -1608.
 -4040. -7667. 5355. 1663.
 4512. ]
 [ 1952. 3989. 6279. 9487.
 681. -2854. -11043. 1260.
 -1608. -4040. -7667. 5355.
 1663. ]
 [ 5724.88631546 1952. 3989. 6279.
 9487. 681. -2854. -11043.
 1260. -1608. -4040. -7667.
 5355. ]
 [ -6066.76862288 4955. 1952. 3989.
 6279. 9487. 681. -2854.
 -11043. 1260. -1608. -4040.
 -7667. ]
 [ -4779.84283715 -7553. 4955. 1952.
 3989. 6279. 9487. 681.
 -2854. -11043. 1260. -1608.
 -4040. ]
 [ -639.08131109 -4196. -7553. 4955.
 1952. 3989. 6279. 9487.
 681. -2854. -11043. 1260.
 -1608. ]
 [ 1857.48537362 -2482. -4196. -7553.
 4955. 1952. 3989. 6279.
 9487. 681. -2854. -

In [37]:
whole_df = ks.DataFrame(whole_df, columns=fin_col)
print(whole_df.iloc[-10:])

Difference lag_1 lag_2 lag_3 lag_4 lag_5 lag_6 lag_7 lag_8 lag_9 lag_10 lag_11 lag_12
41 -4779.842837 -7553.000000 4955.000000 1952.000000 3989.000000 6279.000000 9487.000000 681.000000 -2854.000000 -11043.000000 1260.000000 -1608.000000 -4040.000000
42 -639.081311 -4196.000000 -7553.000000 4955.000000 1952.000000 3989.000000 6279.000000 9487.000000 681.000000 -2854.000000 -11043.000000 1260.000000 -1608.000000
43 1857.485374 -2482.000000 -4196.000000 -7553.000000 4955.000000 1952.000000 3989.000000 6279.000000 9487.000000 681.000000 -2854.000000 -11043.000000 1260.000000
44 -11109.058421 2153.000000 -2482.000000 -4196.000000 -7553.000000 4955.000000 1952.000000 3989.000000 6279.000000 9487.000000 681.000000 -2854.000000 -11043.000000
45 -19933.907739 -11109.058421 1857.485374 -639.081311 -4921.540731 -6179.844609 5349.557707 1660.582372 3652.278418 5891.347611 9028.000000 417.850082 -3038.529654
46 -15232.221745 -19933.907739 -11109.058421 1857.485374 -872.871924 -4921.540731 -6179.844609 5349.557707 1660.582372 3652.278418 5891.347611 9028.000000 417.850082
47 -3479.386032 -15232.221745 -19933.907739 -11109.058421 1568.169801 -872.871924 -4921.540731 -6179.844609 5349.557707 1660.582372 3652.278418 5891.347611 9028.000000
48 1952.746840 -3479.386032 -15232.221745 -19933.907739 -11109.991224 1568.169801 -872.871924 -4921.540731 -6179.844609 5349.557707 1660.582372 3652.278418 5891.347611
49 6204.967965 1952.746840 -3479.386032 -15232.221745 -19738.571241 -11109.991224 1568.169801 -872.871924 -4921.540731 -6179.844609 5349.557707 1660.582372 3652.278418
50 6143.445941 6204.967965 1952.746840 -3479.386032 -15141.453222 -19738.571241 -11109.991224 1568.169801 -872.871924 -4921.540731 -6179.844609 5349.557707 1660.582372

In [38]:
ks.set_option('compute.ops_on_diff_frames', True)
whole_df['Order_Date_Id'] = main_df.iloc[:,1]
whole_df['Sales'] = main_df.iloc[:,2]
whole_df = whole_df.sort_values(by=['Order_Date_Id'], ascending=True)
print(whole_df.iloc[-10:])
ks.reset_option('compute.ops_on_diff_frames')

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-4096074413772337> in <module> 
 1 ks . set_option ( 'compute.ops_on_diff_frames' , True ) 
 ----> 2 whole_df [ 'Order_Date_Id' ] = main_df . iloc [ : , 1 ] 
 3 whole_df [ 'Sales' ] = main_df . iloc [ : , 2 ] 
 4 whole_df = whole_df . sort_values ( by = [ 'Order_Date_Id' ] , ascending = True ) 
 5 print ( whole_df . iloc [ - 10 : ] ) 

 /databricks/python/lib/python3.7/site-packages/databricks/koalas/indexing.py in __getitem__ (self, key) 
 249 
 250 cond , limit , remaining_index = self . _select_rows ( rows_sel ) 
 --> 251 column_labels , data_spark_columns , returns_series = self . _select_cols ( cols_sel ) 
 252 
 253 if cond is None and limit is None and returns_series : 

 /databricks/python/lib/python3.7/site-packages/databricks/koalas/indexing.py in _select_cols (self, cols_sel, missing_keys) 
 1227 data_spark_columns = cols_sel . _internal . data_spark_columns
 1228 elif isinstance ( cols_sel , int ) : 
 -> 1229 if cols_sel > len ( self . _internal . column_labels ) : 
 1230 raise KeyError ( cols_sel ) 
 1231 column_labels = [ self . _internal . column_labels [ cols_sel ] ] 

 /databricks/python/lib/python3.7/site-packages/databricks/koalas/utils.py in _lazy_property (self) 
 413 def _lazy_property ( self ) : 
 414 if not hasattr ( self , attr_name ) : 
 --> 415 setattr ( self , attr_name , fn ( self ) ) 
 416 return getattr ( self , attr_name ) 
 417 

 /databricks/python/lib/python3.7/site-packages/databricks/koalas/indexing.py in _internal (self) 
 1090 internal = super ( iLocIndexer , self ) . _internal
 1091 sdf = _InternalFrame.attach_distributed_sequence_column(
 -> 1092 internal . spark_frame , column_name = self . _sequence_col
 1093 )
 1094 return internal . copy ( spark_frame = sdf . orderBy ( NATURAL_ORDER_COLUMN_NAME ) ) 

 /databricks/python/lib/python3.7/site-packages/databricks/koalas/internal.py in attach_distributed_sequence_column (sdf, column_name) 
 641 counts = map(
 642 lambda x : ( x [ "key" ] , x [ "count" ] ) , 
 --> 643 sdf . groupby ( sdf [ spark_partition_column ] . alias ( "key" ) ) . count ( ) . collect ( ) , 
 644 )
 645 

 /databricks/spark/python/pyspark/sql/dataframe.py in collect (self) 
 550 # Default path used in OSS Spark / for non-DF-ACL clusters: 
 551 with SCCallSiteSync ( self . _sc ) as css : 
 --> 552 sock_info = self . _jdf . collectToPython ( ) 
 553 return list ( _load_from_socket ( sock_info , BatchedSerializer ( PickleSerializer ( ) ) ) ) 
 554 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o7097.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 26 in stage 449.0 failed 1 times, most recent failure: Lost task 26.0 in stage 449.0 (TID 29424, localhost, executor driver): ExecutorLostFailure (executor driver exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 256225 ms
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:2362)
	at org.apac

In [39]:
order_date_id = list(whole_df['Order_Date_Id'].to_numpy())
print(order_date_id)

In [40]:
from dateutil.relativedelta import relativedelta
order_date_id2 = order_date_id
for i in range(len(order_date_id)-num_predictions,len(order_date_id)):
    #print(order_date_id2[i])
    add_month = relativedelta(months=1)
    new_month = order_date_id2[i-1] + add_month
    order_date_id2[i] = new_month
print(order_date_id2)

In [41]:
# Attach this list in the dataframe
# Drop that column
whole_df.drop('Order_Date_Id', axis = 1, inplace = True)
whole_df['Order_Date_Id'] = order_date_id2
print(whole_df.tail(10))

In [42]:
sale_list = whole_df['Sales'].to_list()
difference_list = whole_df['Difference'].to_list()
print(sale_list)
print(difference_list)

In [43]:
for i in range(len(difference_list)-num_predictions,len(difference_list)):
    sale_list[i] = sale_list[i-1] + difference_list[i]
print(sale_list)

In [44]:
# Attach this list in the dataframe
# Drop that column
whole_df.drop('Sales', axis = 1, inplace = True)
whole_df['Sales'] = sale_list
print(whole_df.tail(10))

In [45]:
# Re-arranging the columns in df
fin_col = ['Order_Date_Id', 'Sales'] + fin_col
whole_df = whole_df[fin_col]
print(whole_df.tail(20))

In [46]:
# Plot actual and predicted
plot_data = [
    go.Scatter(
        x=whole_df['Order_Date_Id'],
        y=whole_df['Sales'],
        name='actual'
    ),
        go.Scatter(
        x=whole_df['Order_Date_Id'].iloc[-num_predictions-1:],
        y=whole_df['Sales'].iloc[-num_predictions-1:],
        name='predicted'
    )
    
]
plot_layout = go.Layout(
        title='Sales Prediction'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)